# OCHL generator

## import functions

In [3]:
!pip install plotly==4.8

You should consider upgrading via the 'c:\users\ardit\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
from scipy.stats import skewnorm
import plotly.graph_objects as go
import plotly.express as px
import pandas_ta as ta
import pandas as pd
import numpy as np
import random

#normal distribution for difference between low and high
def create_pdf(sd, mean, alfa):
    #invertire il segno di alfa
    x = skewnorm.rvs(alfa, size=1000000) 
    print(x)
    def calc(k, sd, mean):
        return (k*sd)+mean
    x = calc(x, sd, mean) #standard distribution
    return x

def graph_stock(df_original):
  #time_column is a string
  df = df_original.copy()
  pd.options.plotting.backend = "plotly"
  #i need to create a column with the index, otherwise plotly won't graph it
  df['date'] = df.index
  #fig = px.line(df, x='time', y=['price', 'price_2'])
  fig = px.line(df, x='date', y=df.columns)
  fig.show()
  #save to png
  #fig.write_image("csvfiles/btc.mark2mkt.png")

def graph_OCHL(df_OCHL, title):
    #fig_1 = px.line(df, x=df.index, y=df.columns, title=title)
    fig_2 = go.Figure(data=go.Ohlc(x=df_OCHL.index,
                        open=df_OCHL['open'],
                        high=df_OCHL['high'],
                        low=df_OCHL['low'],
                        close=df_OCHL['close']))
    fig_2.update(layout_xaxis_rangeslider_visible=False)
    fig_2.show()

def OCHL(group_values):
    min_ = min(group_values)
    max_ = max(group_values)
    range = max_ - min_
    open = min_+range*random.random()
    close = min_+range*random.random()
    return min_, max_, open, close

def simulate_stock(initial_price, drift, volatility, trend, days):
    days=days*24
    def create_pdf(sd, mean, alfa):
        #invertire il segno di alfa
        x = skewnorm.rvs(alfa, size=1000000) 
        def calc(k, sd, mean):
            return (k*sd)+mean
        x = calc(x, sd, mean) #standard distribution
        return x

    def create_empty_df(days):
        #creare un empty DataFrame con le date
        empty = pd.DatetimeIndex(
            pd.date_range("2020-01-01", periods=days, freq="D")
        )
        empty = pd.DataFrame(empty)
        #si tagliano ore, minuti, secondi
        empty

        #si tagliano ore, minuti, secondi
        empty.index = [str(x)[0:empty.shape[0]] for x in list(empty.pop(0))]
        empty

        #final dataset con values
        stock = pd.DataFrame([x for x in range(0, empty.shape[0])])
        stock.index = empty.index
        return stock

    #ran = create_pdf(0.1, 0.2, 0) #del?
    #np.random.choice(ran) #del?
    #skeleton
    stock = create_empty_df(days)
    #initial price
    stock[0][0] = initial_price
    #create entire stock DataFrame
    x = create_pdf(volatility, drift, trend)
    for _ in range(1, stock.shape[0]):
        stock.iloc[_] = stock.iloc[_-1]*(1+np.random.choice(x))
    stock.index = pd.DatetimeIndex(stock.index)
    return stock

def simulate_OCHL_stock(df, graph_timeseries=False, graph_OCHL=False):
    df_ = list()
    #df.groupby(np.arange(len(df))//24).apply(OCHL) non funziona
    #sarebbe il modo corretto, ma devo creare un nuovo df da 0
    for a, b in df.groupby(np.arange(len(df))//24):
        group_values = np.array(b.values).flatten()
        low, high, open, close = OCHL(group_values)
        df_.append([low, high, open, close])

    #
    df_OCHL = pd.DataFrame(df_, index=pd.Series(pd.date_range("2020-01-01", periods=365, freq="D")), columns=['low', 'high', 'open', 'close'])

    #graph

    if graph_timeseries==True: graph_stock(df)
    if graph_OCHL==True:
        fig = go.Figure(
            data=go.Ohlc(x=df_OCHL.index,
            open=df_OCHL['open'],
            high=df_OCHL['high'],
            low=df_OCHL['low'],
            close=df_OCHL['close'])
        )
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show()
    return df_OCHL

## generate stock

In [11]:
df_OCHL = simulate_OCHL_stock(simulate_stock(1000, 0, 0.01, 0, 365), graph_timeseries=True, graph_OCHL=True)
df_OCHL

,low,high,open,close
2020-01-01,873.775745,1000.000000,904.744957,903.140216
2020-01-02,879.127661,916.348766,889.944237,907.994218
2020-01-03,868.427811,916.285488,891.589540,877.236152
2020-01-04,868.515751,928.201578,889.270305,886.181457
2020-01-05,845.909331,899.989786,871.267375,853.319731
...,...,...,...,...
2020-12-26,381.552876,405.804907,383.143091,394.717545
2020-12-27,365.167644,381.355161,370.595889,365.754583
2020-12-28,355.094880,376.792687,357.686178,366.237202
2020-12-29,349.675093,372.413069,372.022256,350.410608


## create backup

In [6]:
df_backup = df_OCHL.copy()

# pandas_ta

In [ ]:
!pip install pandas_ta

## strategy_1

In [14]:
#import backup
df = df_backup.copy()

MyStrategy = ta.Strategy(
    name="DCSMA10",
    ta=[
        {"kind": "ohlc4"},
        {"kind": "sma", "length": 10},
        {"kind": "sma", "length": 20},
    ]
)

# (2) Run the Strategy
df.ta.strategy(MyStrategy)
df = df.drop(['low', 'high', 'open'], axis=1)
graph_stock(df)

## strategy_2

In [13]:
#import backup
df = df_backup.copy()

MyStrategy = ta.Strategy(
    name="strategy_3",
    ta=[{
        "close": 'close', 
        "kind": "bbands", 
        "length": 15,
        "std": 2
    }]
)

# (2) Run the Strategy
df.ta.strategy(MyStrategy)
df = df.drop(['low', 'high', 'open'], axis=1)
graph_stock(df)